In [ ]:
# GHOST - Example usage (solving the 2D linear advection equation)

import sys  
import numpy as np
sys.path.insert(0, './src')
from Mesh import Mesh2D
from Solver import Solver

# read in mesh in GMSH format
mesh = Mesh2D("advection_demo", "./example_mesh_4x4.msh")

# set up periodic boundary conditions
left = np.array([1.0,0.0,0.0]) 
right = np.array([1.0,0.0,1.0])
bottom = np.array([0.0,1.0,0.0])
top = np.array([0.0,1.0,1.0])
mesh.add_bc_on_hyperplanes([left,right,bottom,top],[1,2,3,4])
mesh.make_periodic((1,2),[1]) # left-right periodic (bcs parallel to axis 1)
mesh.make_periodic((3,4),[0]) # top-bottom periodic (axis 0)

# set volume and facet quadrature degrees
p = 3
volume_quadrature_degree = 2*p
facet_quadrature_degree = 2*p+1

# set wave angle and speed
theta = np.pi/4
a = np.sqrt(2)

# input solver parameters
params = {"project_title": "advection_demo",
         "problem": "constant_advection",
         "initial_condition": "sine",
         "wavelength": np.ones(2),
         "wave_speed": a*np.array([np.sin(theta),np.cos(theta)]),
         "upwind_parameter": 0.0,
         "integration_type": "quadrature",
         "solution_degree": 3,
         "volume_quadrature_degree": volume_quadrature_degree,
         "facet_quadrature_degree": facet_quadrature_degree,
         "solution_representation": "modal",
         "form": "weak",
         "correction": "c_dg",
         "time_integrator": "rk44",
         "final_time": 1.0,
         "time_step_scale": 0.0025}

# set up solver and pre-process simulation
advection_solver = Solver(params,mesh)

In [ ]:
# run the solver using the specified parameters and mesh
advection_solver.run()

In [ ]:
# post-process and generate plots of numerical and exact solution
advection.post_process()
advection.plot(plot_curves=True,plot_exact=True)